### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/medical_image_processing/

Mounted at /content/gdrive
/content/gdrive/MyDrive/medical_image_processing


In [ ]:
from fundal_image.utils import create_dir, get_filenames_sorted
from patches import generate_patches_images_only, reconstruct_patches
from metrics import iou, dice_coef, dice_loss, DiceBCELoss, bce
from metrics import compute_metrics, save_results
from utils import read_image_train, read_mask_train

In [ ]:
image_sizes = {"HRF": (2336, 3504), "drive": (584, 565), "chase_db": (960, 999)}
patch_size = {"HRF": (768, 768),
              "drive":(128, 128),
              "chase_db":(128, 128)
              }

dataset_name = "chase_db"
patch_size = patch_size[dataset_name]
patch_height = patch_size[0]
patch_width = patch_size[1]

In [ ]:
loss_str = "dice_loss"
with_augmentation = False
stride=128
seed = 42
run_id = 1
test_percentage = 0.4
class_weight = 0.4
class_weight_str = str(class_weight).replace(".", "_")
aug_string = ""
if with_augmentation:
    aug_string = "_hvr"
test_percentage_str = str(test_percentage).replace(".", "_")
root_path = f"/content/datasets/{dataset_name}_{patch_height}_{stride}_{loss_str}_{class_weight_str}_{aug_string}_{test_percentage_str}_{run_id}/"

images_path_train = f"{root_path}/training/images/"
gt_path_train = f"{root_path}/training/1st_manual/"
images_path_test = f"{root_path}/test/images/"
gt_path_test = f"{root_path}/test/1st_manual/"

create_dir(images_path_train)
create_dir(gt_path_train)
create_dir(images_path_test)
create_dir(gt_path_test)

In [ ]:
def wbce(actual, predicted):
    y_pred = K.clip(predicted, K.epsilon(), 1 - K.epsilon())
    term_0 = (1 - actual) * K.log(1 - y_pred)
    term_1 = class_weight * actual * K.log(y_pred + K.epsilon())
    return -K.mean(term_0 + term_1, axis=0)

In [ ]:
root_path

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1/'

In [ ]:
images_path_train

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/'

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/chase_db/split_1/training/images/*.* {images_path_train}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/chase_db/split_1//training/groundtruth1/*.* {gt_path_train}

In [ ]:
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/chase_db/split_1/test/images/*.* {images_path_test}
!cp /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets/chase_db/split_1/test/groundtruth1/*.* {gt_path_test}

In [ ]:
!ls /content/gdrive/MyDrive/medical_image_processing/fundal_image/datasets//chase_db/split_1/training/images

Image_01L.jpg  Image_03R.jpg  Image_06L.jpg  Image_08R.jpg  Image_13R.jpg
Image_01R.jpg  Image_04L.jpg  Image_06R.jpg  Image_09L.jpg  Image_14L.jpg
Image_02L.jpg  Image_04R.jpg  Image_07L.jpg  Image_09R.jpg  Image_14R.jpg
Image_02R.jpg  Image_05L.jpg  Image_07R.jpg  Image_10L.jpg
Image_03L.jpg  Image_05R.jpg  Image_08L.jpg  Image_10R.jpg


In [ ]:
# images_files, gt_files = get_filenames_sorted(images_path, gt_path)
# print(len(images_files), len(gt_files))
# # train_test_split
# from sklearn.model_selection import train_test_split
# label = [_xtrain.rsplit("/", 1)[1].rsplit(".", 1)[0].rsplit("_", 1)[1] for _xtrain in images_files] 
# xtrain, xtest, ytrain, ytest = train_test_split(images_files, gt_files, stratify=label,  test_size=test_percentage, random_state=seed)

In [ ]:
xtrain, ytrain = get_filenames_sorted(images_path_train, gt_path_train)
xtest, ytest = get_filenames_sorted(images_path_test, gt_path_test)

In [ ]:
print(len(xtrain), len(xtest))
# print(label)

23 5


In [ ]:
xtrain[:5]

['/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/Image_01L.jpg',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/Image_01R.jpg',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/Image_02L.jpg',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/Image_02R.jpg',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/images/Image_03L.jpg']

In [ ]:
print(len(ytrain), len(ytest))
ytrain[:5]

23 5


['/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/1st_manual/Image_01L_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/1st_manual/Image_01R_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/1st_manual/Image_02L_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/1st_manual/Image_02R_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//training/1st_manual/Image_03L_1stHO.png']

In [ ]:
ytest[:5]

['/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//test/1st_manual/Image_11L_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//test/1st_manual/Image_11R_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//test/1st_manual/Image_12L_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//test/1st_manual/Image_12R_1stHO.png',
 '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//test/1st_manual/Image_13L_1stHO.png']

In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
from sklearn.feature_extraction import image
import numpy as np
from PIL import Image
from tqdm import tqdm
import pandas as pd
import cv2

from fundal_image.utils import  create_dir
from utils import get_filenames_sorted

def generate_patches_images_only(image_files,
                                 save_path_image,
                                 max_patch=30,
                                 dataset_name="",
                                 patch_size=(512, 512),
                                 stride=None,
                                 padding="zeros",
                                 exclude_boundary=True,
                                 zero_padding_size=(0, 0),
                                 gt_files=None,
                                 save_path_gt=None
                                 ):
    
    def patches_with_stride(image_filename, im, _patch_size, _stride, gt_name=None, gt=None):
        patches = []
        gt_patches = []
        # print("Image size", im.shape)
        metadata_list = []
        columns = ["image_filename", "row", "col", "patch_height", "patch_width", "stride", "num_fg_pixels"]
        if padding == "zeros":
            num_zero_padding_height = im.shape[0] % patch_size[0]
            num_zero_padding_width = im.shape[1] % patch_size[1]

            if num_zero_padding_height > 0:
                num_zero_padding_height = _patch_size[0] - num_zero_padding_height

            if num_zero_padding_width > 0:
                num_zero_padding_width = _patch_size[1] - num_zero_padding_width

            height_odd = 0 if (num_zero_padding_height % 2) == 0 else 1
            width_odd = 0 if (num_zero_padding_width % 2) == 0 else 1
        else:
            raise Exception(f"Value for argument padding is not mentioned")

        # print(f"Image shape {im.shape}. Patch size {_patch_size}.  Num Zero padding rows {num_zero_padding_height}. Num zero padding cols {num_zero_padding_width}")
        im = np.pad(im, ((num_zero_padding_height // 2, num_zero_padding_height // 2 + height_odd),
                                (num_zero_padding_width // 2, num_zero_padding_width // 2 + width_odd),
                                (0, 0)
                                )
                           )
        if gt is not None:
            if len(gt.shape) == 3:
                gt = np.pad(gt, ((num_zero_padding_height // 2, num_zero_padding_height // 2 + height_odd),
                                 (num_zero_padding_width // 2, num_zero_padding_width // 2 + width_odd),
                                 (0, 0)
                                 )
                            )
            else:
                gt = np.pad(gt, ((num_zero_padding_height // 2, num_zero_padding_height // 2 + height_odd),
                                 (num_zero_padding_width // 2, num_zero_padding_width // 2 + width_odd)
                                 )
                            )

        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # print(f"image shape after padding {im.shape}")
        for row in range(0, im.shape[0], _stride):
            for col in range(0, im.shape[1], _stride):
                if row + _patch_size[0] >= im.shape[0]:
                    row = im.shape[0] - _patch_size[0]
                if col + _patch_size[1] >= im.shape[1]:
                    col = im.shape[1] - _patch_size[1]
                image_patch = im[row:row + _patch_size[0], col:col + _patch_size[1]]
                num_fg_pixels = np.sum(im_gray[row:row + _patch_size[0], col:col + _patch_size[1]] > 0)
                if gt is not None:
                    gt_patch = gt[row:row + _patch_size[0], col:col + _patch_size[1]]
                if num_fg_pixels > 0:
                    patches.append(image_patch)
                    metadata_list.append([image_filename, row, col, _patch_size[0], _patch_size[1], _stride, num_fg_pixels])
                    if gt is not None:
                        gt_patches.append(gt_patch)


        metadata_df = pd.DataFrame(metadata_list, columns=columns )
        patches_np = np.asarray(patches)
        if gt is not None:
            gt_patches_np = np.asarray(gt_patches)
        else:
            gt_patches_np = None

        return patches_np, metadata_df, gt_patches_np

    num_images = len(image_files)
    print(f"Generating patches for {num_images} images")
    metadata_df = None
    create_dir(save_path_image)
    im_patches_dict = dict()
    gt_patches_dict = dict()
    for idx, image_filename in tqdm(enumerate(image_files), total=len(image_files)):
        image_filename = image_filename.replace("\\", "/")
        # print("Image file", image_filename)
        name = image_filename.rsplit("/", 1)[1].rsplit(".", 1)[0]
        # image1 = Image.open(x)
        # image_array = np.asarray(image1)
        image_array = cv2.imread(image_filename)

        if gt_files is not None and len(gt_files) > 0:
            gt_filename = gt_files[idx]
            gt_filename = gt_filename.replace("\\", "/")
            # print("Image file", image_filename)
            gt_name = gt_filename.rsplit("/", 1)[1].rsplit(".", 1)[0]


            gt = cv2.imread(gt_filename)
            # gt = np.asarray(Image.open(gt_filename))
            # print(gt.dtype)
            if len(gt.shape) == 3:
                gt = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)

        if not exclude_boundary:
            # add zero padding
            image_array = np.pad(image_array, zero_padding_size)
            gt = np.pad(gt, zero_padding_size)

        if stride is None:
            image_patches = image.extract_patches_2d(image_array, patch_size, max_patches=max_patch, random_state=42)
        else:
            ret_value = patches_with_stride(image_filename, image_array, patch_size, stride, gt_name, gt)
            if ret_value is None :
                raise Exception(f"Could not create patches for {image_filename}")
            else:
                image_patches, _metadata_df, gt_patches = ret_value[0], ret_value[1], ret_value[2]
            if metadata_df is not None:
                metadata_df = metadata_df.append(_metadata_df)
            else:
                metadata_df = _metadata_df
        im_patches_dict[name] = image_patches
        gt_patches_dict[name] = gt_patches

        # print(gt_patches.dtype)


        if save_path_image is not None and len(save_path_image) > 0:
            count = 0
            for i in range(len(image_patches)):
                # data = Image.fromarray(image_patches[i])
                # print(save_path_image, dataset_name, name)
                image_path_file = f"{save_path_image}/{dataset_name}_{name}_{count}.png"
                # print(f"Saving patches to {image_path_file}")
                cv2.imwrite(image_path_file, image_patches[i])

                if gt_patches is not None and save_path_gt is not None:
                    gt_path_file = f"{save_path_gt}/{dataset_name}_{name}_{count}.png"
                    # print(gt_path_file)
                    # print(type(gt_patches))
                    # print(gt_patches[i].shape)
                    # print(gt_patches[i].dtype)
                    # print(np.min(gt_patches[i]), np.max(gt_patches[i]))
                    cv2.imwrite(gt_path_file, gt_patches[i])
                count += 1

    return im_patches_dict, metadata_df, gt_patches_dict

In [ ]:
num_pixels_to_add = [0, 0]
if (image_sizes[dataset_name][0] % patch_size[0]) > 0:
    num_pixels_to_add[0] = ((image_sizes["chase_db"][0] // patch_size[0]) + 1) * patch_size[0]
    num_pixels_to_add[1] = ((image_sizes["chase_db"][1] // patch_size[1]) + 1) * patch_size[1]

split_name = "split_2"
data_path = root_path
splits =["train","test"]


split = "train"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_train, metadata_train_df, gt_patches_dict_train = generate_patches_images_only(xtrain,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytrain,
                                                                save_path_gt=gt_patches_path)
metadata_train_df.to_csv(f"{data_path}/{split_name}/image_patches_{stride}_{stride}_{split}.csv", index=False)



Generating patches for 23 images


100%|██████████| 23/23 [00:01<00:00, 14.80it/s]


In [ ]:
gt_patches_path

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/train/mask/'

In [ ]:
split = "test"
#for split in splits:
image_patches_path = f"{data_path}/{split_name}/{split}/image/"
create_dir(image_patches_path)
gt_patches_path = f"{data_path}/{split_name}/{split}/mask/"
create_dir(gt_patches_path)
im_patches_dict_test, metadata_test_df, gt_patches_dict_test = generate_patches_images_only(xtest,
                                                                image_patches_path,
                                                                patch_size=patch_size,
                                                                stride=stride,
                                                                zero_padding_size=num_pixels_to_add,
                                                                gt_files=ytest,
                                                                save_path_gt=gt_patches_path)
metadata_test_df.to_csv(f"{data_path}/{split_name}/image_patches_{patch_height}_{stride}_{split}.csv", index=False)


Generating patches for 5 images


100%|██████████| 5/5 [00:00<00:00, 16.45it/s]


In [ ]:
metadata_test_df.shape

(296, 7)

In [ ]:
metadata_train_df.shape

(1337, 7)

In [ ]:
metadata_train_df.head()

,image_filename,row,col,patch_height,patch_width,stride,num_fg_pixels
0,/content/datasets/chase_db_128_128_dice_loss_0...,0,128,128,128,128,72
1,/content/datasets/chase_db_128_128_dice_loss_0...,0,256,128,128,128,5537
2,/content/datasets/chase_db_128_128_dice_loss_0...,0,384,128,128,128,9894
3,/content/datasets/chase_db_128_128_dice_loss_0...,0,512,128,128,128,9705
4,/content/datasets/chase_db_128_128_dice_loss_0...,0,640,128,128,128,4833


In [ ]:
import glob
split = "test"
image_patches_path = f"{data_path}/{split_name}/{split}/image/"

reconstructed_original_path = f"{data_path}/{split_name}/reconstructed/{split}/image"
create_dir(reconstructed_original_path)
reconstruct_patches(im_patches_dict_test,
                    metadata_test_df,
                    reconstructed_original_path
                    )


/content/gdrive/MyDrive/medical_image_processing/patches.py:294: RuntimeWarning: invalid value encountered in true_divide
  im[:, :, channel] = np.true_divide(im[:, :, channel], count_im)


In [ ]:
print(list(gt_patches_dict_test.keys()))

['Image_11L', 'Image_11R', 'Image_12L', 'Image_12R', 'Image_13L']


In [ ]:
type(gt_patches_dict_test["Image_11L"])

numpy.ndarray

In [ ]:
reconstructed_gt_path = f"{data_path}/{split_name}/reconstructed/{split}/mask"

create_dir(reconstructed_gt_path)
reconstruct_patches(gt_patches_dict_test,
                    metadata_test_df,
                    reconstructed_gt_path,
                    )


/content/gdrive/MyDrive/medical_image_processing/patches.py:296: RuntimeWarning: invalid value encountered in true_divide
  im[:, :] = np.true_divide(im[:, :], count_im)


In [ ]:
print([k for k in im_patches_dict_test.keys()])

['Image_11L', 'Image_11R', 'Image_12L', 'Image_12R', 'Image_13L']


### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def DiceBCELoss(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(inputs * targets)
    dice_loss = 1 - (2.*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets) + dice_loss

    return Dice_BCE

def bce(y_true, y_pred):
    
    inputs = tf.keras.layers.Flatten()(y_true)
    targets = tf.keras.layers.Flatten()(y_pred)
    BCE = tf.keras.losses.BinaryCrossentropy()
    Dice_BCE = BCE(inputs, targets)

    return Dice_BCE



### Training the Model:

In [ ]:
if loss_str == "bce":
    loss = bce
elif loss_str == "dice_bce":
    loss = DiceBCELoss
elif loss_str == "dice_loss":
    loss = dice_loss
elif loss_str == "wbce":
    loss = wbce
else:
    raise Exception("Invalid value for loss_str")
    
print(loss)

<function dice_loss at 0x7f0cd02447a0>


In [ ]:
H = patch_height
W = patch_width

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
from keras.metrics import AUC

In [ ]:
model_dir = f"{root_path}/trained_models/"

In [ ]:
create_dir(model_dir)

In [ ]:
!ls {model_dir}

In [ ]:
%cd /content/datasets

/content/datasets


In [ ]:
from glob import glob

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir(model_dir)

""" Hyperparameters """
batch_size = 8
lr = 1e-4
num_epochs = 50
model_path = os.path.join(f"{model_dir}", "model.h5")
csv_path = os.path.join(f"{model_dir}", "data.csv")

""" Dataset """
dataset_path = f"{data_path}/{split_name}"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
# train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
model.compile(loss=loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision(), AUC()])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 1337 - 1337
Valid: 296 - 296
Epoch 1/50
168/168 [==============================] - ETA: 0s - loss: 0.6020 - dice_coef: 0.3969 - iou: 0.2553 - recall: 0.7805 - precision: 0.3219 - auc: 0.9039
Epoch 1: val_loss improved from inf to 0.91679, saving model to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//trained_models/model.h5
168/168 [==============================] - 25s 64ms/step - loss: 0.6020 - dice_coef: 0.3969 - iou: 0.2553 - recall: 0.7805 - precision: 0.3219 - auc: 0.9039 - val_loss: 0.9168 - val_dice_coef: 0.0832 - val_iou: 0.0436 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_auc: 0.3844 - lr: 1.0000e-04
Epoch 2/50
167/168 [============================>.] - ETA: 0s - loss: 0.4372 - dice_coef: 0.5628 - iou: 0.3974 - recall: 0.7790 - precision: 0.6464 - auc: 0.9398
Epoch 2: val_loss improved from 0.91679 to 0.88129, saving model to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//trained_models/model.h5
168/168 [==============================]

### Evaluating the Model:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.models import load_model

In [ ]:

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.png")))
    y = sorted(glob(os.path.join(path, "mask", "*.png")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [ ]:
import glob
supported_extensions = ["png", "jpg", "tif", "jpeg", "gif", "bmp"]

def image_files(path, file_prefix=None):
    all_image_files = []
    # print("path", path)
    # print("file_prefix", file_prefix)
    if file_prefix is None:
        file_names = list(glob.glob(path + "/*.*" ))
    else:
        prefix___ = path + "/" + file_prefix + ".*"
        #print("prefix", prefix___)
        file_names = list(glob.glob(prefix___))

    # print("filenames", file_names)

    for filename in file_names:
        ext = filename.rsplit(".", 1)[1]
        if ext.lower() in supported_extensions:
            all_image_files.append(filename)
    # print("All Image files", all_image_files)
    return sorted(all_image_files)


In [ ]:
results_path = f"{root_path}/results/"
prediction_path = f"{root_path}/predictions/"
prediction_np_path = f"{root_path}/predictions_np/"
create_dir(prediction_path)
create_dir(prediction_np_path)
create_dir(results_path)

In [ ]:
results_path

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results/'

In [ ]:
from glob import glob

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_auc_score
# prediction_path = "predictions"

""" Save the results in this folder """
# create_dir("results_sunil_test")
# create_dir(prediction_path)

""" Load the model """
# with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
model_path = os.path.join(f"{model_dir}", "model.h5")

model = load_model(model_path, compile=False)

""" Load the dataset """
dataset_path = os.path.join(f"{data_path}/{split_name}", "test")
test_x, test_y = load_data(dataset_path)

""" Make the prediction and calculate the metrics values """
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.split("/")[-1].split(".")[0]

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    y = y.astype(np.int)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    np.save(f"{prediction_np_path}/{name}.npy", y_pred)
    y_prob = deepcopy(y_pred).flatten()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.int32)
    y_pred = np.squeeze(y_pred, axis=-1)

    """ Saving the images """
    save_image_path = f"{results_path}/{name}.png"
#    print(ori_x.shape, ori_y.shape, y_pred.shape)
    save_results(ori_x, ori_y, y_pred, save_image_path)
    cv2.imwrite(f"{prediction_path}/{name}.png", y_pred * 255)
    

    """ Flatten the array """
    y = y.flatten()
    y_pred = y_pred.flatten()

    """ Calculate the metrics """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    if np.sum(y) == 0 or np.sum(y) == H * W:
        auc_value = 1
    else:
        auc_value = roc_auc_score(y, y_prob)
    dice_value = dice_coef(y.astype(np.float64), y_prob)

    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value])

score = [s[1:] for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")
print(f"AUC: {score[5]:0.5f}")
print(f"Dice Coeff: {score[6]:0.5f}")
""" Saving """
df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coeff"])
df.to_csv(f"{results_path}/score.csv")

  0%|          | 0/296 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedM

Accuracy: 0.96841
F1: 0.42732
Jaccard: 0.33613
Recall: 0.41827
Precision: 0.51050
AUC: 0.84659
Dice Coeff: 0.42109


# Reconstruct original image

In [ ]:
prediction_np_path

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/'

In [ ]:
from glob import glob

In [ ]:
# Create patches dictionary
predicted_patches_dict = dict()
for file in glob(f"{prediction_np_path}/*.npy"):
    print("file", file)
    patch = np.load(file)
    fname_without_ext = file.rsplit("/", 1)[1].rsplit(".", 1)[0]
    fname_without_ext = fname_without_ext[1:]
    patch_number = int(fname_without_ext.rsplit("_", 1)[1])
    image_file = fname_without_ext.rsplit("_", 1)[0]
    print(image_file, patch_number )

    if image_file in predicted_patches_dict.keys():
        predicted_patches_dict[image_file][patch_number] = np.load(file)
    else:
        patches = [None] * 140
        patches[patch_number] = np.load(file)
        predicted_patches_dict[image_file] = patches

file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11L_5.npy
Image_11L 5
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_13L_53.npy
Image_13L 53
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11R_34.npy
Image_11R 34
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11L_58.npy
Image_11L 58
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_12L_9.npy
Image_12L 9
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11R_33.npy
Image_11R 33
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11L_54.npy
Image_11L 54
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_11R_59.npy
Image_11R 59
file /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//predictions_np/_Image_12R_34.npy
Image_12R 34
file /content/datasets/chase_db_

In [ ]:
reconstructed_predictions_path = f"{data_path}/{split_name}/reconstructed/{split}/predictions_reconstructed"
create_dir(reconstructed_predictions_path)
reconstruct_patches(predicted_patches_dict,
                    metadata_test_df,
                    reconstructed_predictions_path,
                    mfactor=255
                    )


/content/gdrive/MyDrive/medical_image_processing/patches.py:294: RuntimeWarning: invalid value encountered in true_divide
  im[:, :, channel] = np.true_divide(im[:, :, channel], count_im)


In [ ]:
!pwd

/content/datasets


In [ ]:
def read_mask(path):
    print(path)
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
#    x = x.astype(np.int32)
    return ori_x, x

In [ ]:
def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((ori_x.shape[0], 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    print(f"Saving image of size {cat_images.shape} to {save_image_path}")
    cv2.imwrite(save_image_path, cat_images)

# Reconstruct original image

In [ ]:
results_path_reconstructed_orig_size = f"{root_path}/results_reconstructed_orig_size/"
gt_path_reconstructed_orig_size = f"{root_path}/reconstructed_orig_size/mask/"
im_path_reconstructed_orig_size = f"{root_path}/reconstructed_orig_size/image/"

create_dir(results_path_reconstructed_orig_size)
create_dir(gt_path_reconstructed_orig_size)
create_dir(im_path_reconstructed_orig_size)

In [ ]:
from patches import get_zero_padding_size
num_zero_padding_height, num_zero_padding_width = get_zero_padding_size(patch_size,
                                                                        image_sizes[dataset_name])
height_odd = 0 if (num_zero_padding_height % 2) == 0 else 1
width_odd = 0 if (num_zero_padding_width % 2) == 0 else 1

In [ ]:
reconstructed_pred_files, _ = get_filenames_sorted(reconstructed_predictions_path)
print(len(reconstructed_pred_files))
print(reconstructed_pred_files[:5])
for reconstructed_pred_file in reconstructed_pred_files:
    im = cv2.imread(reconstructed_pred_file)
    name = reconstructed_pred_file.rsplit("/", 1)[1]
    im_cropped = im[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    print(im.shape, im_cropped.shape)
    cv2.imwrite(f"{results_path_reconstructed_orig_size}/{name}", im_cropped)

5
['/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test/predictions_reconstructed/Image_11L.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test/predictions_reconstructed/Image_11R.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test/predictions_reconstructed/Image_12L.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test/predictions_reconstructed/Image_12R.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test/predictions_reconstructed/Image_13L.png']
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)


In [ ]:
gt_path_reconstructed = f"{root_path}/{split_name}/reconstructed/{split}/"

reconstructed_im_files, reconstructed_gt_files = get_filenames_sorted(f"{gt_path_reconstructed}/image/", f"{gt_path_reconstructed}/mask/" )
print(len(reconstructed_gt_files))
print(reconstructed_gt_files[:5])

for reconstructed_im_file, reconstructed_gt_file in zip(reconstructed_im_files, reconstructed_gt_files):
    gt = cv2.imread(reconstructed_gt_file)
    im = cv2.imread(reconstructed_im_file)
    name = reconstructed_gt_file.rsplit("/", 1)[1]
    gt_cropped = gt[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    im_cropped = im[num_zero_padding_height // 2 : - num_zero_padding_height // 2, num_zero_padding_width // 2 : -num_zero_padding_width //2 ]
    print(im.shape, im_cropped.shape)
    print(gt.shape, gt_cropped.shape)

    cv2.imwrite(f"{gt_path_reconstructed_orig_size}/{name}", gt_cropped)
    cv2.imwrite(f"{im_path_reconstructed_orig_size}/{name}", im_cropped)

5
['/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test//mask/Image_11L.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test//mask/Image_11R.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test//mask/Image_12L.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test//mask/Image_12R.png', '/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//split_2/reconstructed/test//mask/Image_13L.png']
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)
(1024, 1024, 3) (960, 999, 3)


In [ ]:
results_path_reconstructed = f"{root_path}/results_reconstructed/"
create_dir(results_path_reconstructed)

In [ ]:
from glob import glob

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, recall_score, precision_score, roc_auc_score

from utils import read_image, read_mask
from utils import get_filenames_sorted
from tqdm import tqdm
from copy import deepcopy
from sklearn.metrics import confusion_matrix
def compute_metrics(data_path: str,
                    results_path: str,
                    threshold: float,
                    model=None,
                    predictions_path: str = None,
                    prediction_np_path=None):
    """ Make the prediction ( if model is not None) and calculate the metrics values. If model is None compute the
    metrics using predicted images present in location predictions_path
    data_path: Path location where images and ground truths are saved. Images should be saved in a folder data_path/image
     and ground truths should be saved at data_path/mask
    result_path: Path to directory where results should be saved

    """

    """ Load the dataset """
    test_x, test_y = get_filenames_sorted(f"{data_path}/image", f"{data_path}/mask")
    SCORE = []
    print("Threshold", threshold)
    print(len(test_x), len(test_y))
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):

        if model is None and predictions_path is None:
            raise Exception("Invalid input parameters. Both model and prediction_path can not be None")
        """ Extracting name """
        name = x.split("/")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)
        y = y.astype(np.int)
#        print(name)

        #     """ Prediction """
        if model != None:
            y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        else:
 #           print("Reading predicted image from ", f"{predictions_path}/{name}.png")
            orig_y_pred, y_pred = read_mask(f"{predictions_path}/{name}.png")
#            print(y_pred.shape, np.max(orig_y_pred), np.min(orig_y_pred))
#        print(y_pred.shape, np.max(y_pred), np.min(y_pred))
        if prediction_np_path is not None:
            np.save(f"{prediction_np_path}/{name}.npy", y_pred)

        y_prob = deepcopy(y_pred).flatten()
        y_pred = y_pred > threshold
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred)

        """ Saving the images """
        save_image_path = f"{results_path}/{name}.png"
 #       print(ori_x.shape, ori_y.shape, y_pred.shape)
        save_results(ori_x, ori_y, y_pred, save_image_path)

        if model is not None:
            cv2.imwrite(f"{predictions_path}/{name}.png", y_pred * 255)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        confusion_metrics = confusion_matrix(y, y_pred, labels=[0, 1])
        if np.sum(y) == 0 or np.sum(y) == y.shape[0] :
            auc_value = 1
        else:
            auc_value = roc_auc_score(y, y_prob)
        dice_value = dice_coef(y.astype(np.float64), y_prob.astype(np.float64))
        SCORE.append([name, threshold, acc_value, f1_value, jac_value, recall_value, precision_value, auc_value, dice_value, confusion_metrics[0,0], confusion_metrics[0,1],confusion_metrics[1,0],confusion_metrics[1,1]])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)

    print(f"Threshold: {threshold:0.5f}")
    print(f"Accuracy: {score[1]:0.5f}")
    print(f"F1: {score[2]:0.5f}")
    print(f"Jaccard: {score[3]:0.5f}")
    print(f"Recall: {score[4]:0.5f}")
    print(f"Precision: {score[5]:0.5f}")
    print(f"AUC: {score[6]:0.5f}")
    print(f"Dice Coeff: {score[7]:0.5f}")


    """ Saving """
    df = pd.DataFrame(SCORE, columns=["Image","Threshold", "Acc", "F1", "Jaccard", "Recall", "Precision", "AUC", "Dice Coef", "TN", "FP","FN","TP"])
    threshold_str = str(threshold).replace(".", "_")
    df.to_csv(f"{results_path}/score_full_{threshold_str}.csv")
    return df, score

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

dataset_path = f"{root_path}/reconstructed_orig_size/"
df = dict()
score = dict()
for threshold in range(4,7):
    df[threshold], score[threshold] = compute_metrics(dataset_path,
                        results_path=results_path_reconstructed,
                        threshold = threshold / 10,
                        model=None,
                        predictions_path=results_path_reconstructed_orig_size,
                        prediction_np_path=None)

Threshold 0.4
5 5


  0%|          | 0/5 [00:00<?, ?it/s]

Saving image of size (960, 3017, 3) to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results_reconstructed//Image_11L.png


 20%|██        | 1/5 [00:01<00:06,  1.72s/it]

Saving image of size (960, 3017, 3) to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results_reconstructed//Image_11R.png


...

Saving image of size (960, 3017, 3) to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results_reconstructed//Image_12R.png


 80%|████████  | 4/5 [00:06<00:01,  1.72s/it]

Saving image of size (960, 3017, 3) to /content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results_reconstructed//Image_13L.png


100%|██████████| 5/5 [00:08<00:00,  1.71s/it]

Threshold: 0.60000
Accuracy: 0.96751
F1: 0.68888
Jaccard: 0.53070
Recall: 0.59381
Precision: 0.84052
AUC: 0.79301
Dice Coeff: 0.68888


In [ ]:
results_path_reconstructed

'/content/datasets/chase_db_128_128_dice_loss_0_4__0_4_1//results_reconstructed/'

In [ ]:
!ls 

chase_db_128_128_dice_loss_0_4__0_4_1  logs


In [ ]:
folder_name = root_path.split("/")[3]
print(folder_name)

chase_db_128_128_dice_loss_0_4__0_4_1


In [ ]:
!zip -r {folder_name}.zip  {folder_name}/

  adding: chase_db_128_128_dice_loss_0_4__0_4_1/ (stored 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/ (stored 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/ (stored 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_12R_1stHO.png (deflated 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_12L_1stHO.png (deflated 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_11R_1stHO.png (deflated 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_13L_1stHO.png (deflated 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_11L_1stHO.png (deflated 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/ (stored 0%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/Image_12R.jpg (deflated 6%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/Image_11L.jpg (deflated 6%)
  adding: chase_db_128_128_dice_loss_0_4__0_4_1/test/imag

In [ ]:
!ls -ltrh


total 384M
drwxr-xr-x  4 root root 4.0K Mar 28 04:50 logs
drwxr-xr-x 12 root root 4.0K Mar 28 04:54 chase_db_128_128_dice_loss_0_4__0_4_1
-rw-r--r--  1 root root 384M Mar 28 04:55 chase_db_128_128_dice_loss_0_4__0_4_1.zip


In [ ]:
!ls /content/gdrive/MyDrive/Blood_Vessel_Results/chase_patches/

In [ ]:
!cp {folder_name}.zip /content/gdrive/MyDrive/Blood_Vessel_Results/chase_patches/

In [ ]:
%cd /content/gdrive/MyDrive/Blood_Vessel_Results/chase_patches/

/content/gdrive/MyDrive/Blood_Vessel_Results/chase_patches


In [ ]:
!pwd

/content/gdrive/MyDrive/Blood_Vessel_Results/chase_patches


In [ ]:
!ls

chase_db_128_128_dice_loss_0_4__0_4_1.zip


In [ ]:
!unzip {folder_name}.zip


Archive:  chase_db_128_128_dice_loss_0_4__0_4_1.zip
   creating: chase_db_128_128_dice_loss_0_4__0_4_1/
   creating: chase_db_128_128_dice_loss_0_4__0_4_1/test/
   creating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_12R_1stHO.png  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_12L_1stHO.png  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_11R_1stHO.png  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_13L_1stHO.png  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/1st_manual/Image_11L_1stHO.png  
   creating: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/Image_12R.jpg  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/Image_11L.jpg  
  inflating: chase_db_128_128_dice_loss_0_4__0_4_1/test/images/Image_11R.jpg  
  inflating: chase_db_128

In [ ]:
drive.flush_and_unmount()